In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.nasnet import NASNetLarge
from keras.applications.resnet_v2 import ResNet152V2

from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parâmetros

In [2]:
# Parâmetros gerais
iteracoes_algoritomo = 10

entrada = '../input/leishmaniose-lamina/'

saida = './leishmaniose/'
train_dir = saida + "train/"
val_dir = saida + "val/"
test_dir = saida + "test/"

batch_size = 10

In [3]:
# leitura dos pesos pré-treinados da ImageNet
# Não inclui as camadas de classificação no topo, ideal para extração de features
models = [
    InceptionV3(weights='imagenet', input_shape=(299, 299, 3), include_top=False),
    Xception(weights='imagenet', input_shape=(299, 299, 3), include_top=False),
    InceptionResNetV2(weights='imagenet', input_shape=(299, 299, 3), include_top=False),
    NASNetLarge(weights='imagenet', input_shape=(331, 331, 3), include_top=False),
    ResNet152V2(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
]

# Reescala dos valores dos pixels do modelo
processamento_input = [
    keras.applications.inception_v3.preprocess_input,
    keras.applications.xception.preprocess_input,
    keras.applications.inception_resnet_v2.preprocess_input,
    keras.applications.nasnet.preprocess_input,
    keras.applications.resnet_v2.preprocess_input
]

234553344/234545216 [==============================] - 2s 0us/step


In [4]:
# Parâmetros treinamento
epocas_treinamento = 20
base_learning_rate = 0.0001

epocas_ajuste_fino = 10
total_epocas = epocas_treinamento + epocas_ajuste_fino

# Funções

In [5]:
# Verifica imagens duplicadas no dataset leishmaniose
from skimage.io import imread

def img_repetidas():
    
    img_positivo = glob(entrada + 'Positivo/*.jpg')
    img_negativo = glob(entrada + 'Negativo/*.jpg')
    img_all = img_positivo + img_negativo

    media_imagem = []

    for i in img_positivo:
        imagem = imread(i)
        media_imagem.append(imagem.mean())

    for j in img_negativo:
        imagem = imread(j)
        media_imagem.append(imagem.mean())

    # Verifica imagens repetidas
    cont = 0
    for i in range(0, len(media_imagem) - 1):
        for j in range (i+1, len(media_imagem)):
            if media_imagem[i] == media_imagem[j]:
                print("[%s] Imagem original: %s - Média: %s" %(i, img_all[i], media_imagem[i]))
                print("[%s] Imagem repetida: %s - Média: %s" %(j, img_all[j], media_imagem[j]))
                print("----------------------------------------------------")
                cont+=1
    print("Quantidade de registros duplicados: ", cont)

In [6]:
# Construir um modelo a partir de redes neurais pré-treinadas
def modelo_base(rede):
    
    base_model = models[rede]
    img_size = (base_model.input.shape[1], base_model.input.shape[2])
    
    # Processa as entradas do modelo
    preprocess_input = processamento_input[rede]
    
    return base_model, img_size, preprocess_input

In [7]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import shutil
import splitfolders

# Divide as imagens em treino, validação e teste
def split_imagens():
    
    # Remove o diretorio caso exista (antes de criá-lo)
    if os.path.exists(saida) and os.path.isdir(saida):
        shutil.rmtree(saida)
    
    # Divisão entre treino, validaçao e teste (60%, 10%, 30%)
    splitfolders.ratio(entrada, output="leishmaniose", seed=None, ratio=(.6, .1, .3), group_prefix=None) # default values

In [9]:
from tensorflow import data

# Leitura dos dados
def leitura_dados(img_size):
    
    # Dados de treino
    train_dataset = image_dataset_from_directory(
        train_dir,
        image_size=img_size,
        batch_size=batch_size)

    # Dados de validação
    validation_dataset = image_dataset_from_directory(
        val_dir,
        image_size=img_size,
        batch_size=batch_size)
    
    # Dados de teste
    test_dataset = image_dataset_from_directory(
        test_dir,
        image_size=img_size,
        batch_size=batch_size)
    
    # Nome da classe
    class_names = train_dataset.class_names
    
    # Nome das imagens
    file_paths_train = train_dataset.file_paths
    file_paths_test = test_dataset.file_paths
    
    # pré-busca em buffer para carregar imagens do disco sem que o I/O se torne um bloqueio
    AUTOTUNE = data.AUTOTUNE
    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
    validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
    
    return train_dataset, validation_dataset, test_dataset, class_names, file_paths_train, file_paths_test

In [10]:
# Verifica duplicação na divisão do dataset entre treino, validação e teste
def duplicacao_treino_teste():
    cont = 0
    for i in range(0, len(file_paths_train)):
        for j in range (0, len(file_paths_test)):
            if file_paths_train[i] == file_paths_test[j]:
                print("[%s] Imagem treino: %s" %(i, file_paths_train[i]))
                print("[%s] Imagem validação: %s" %(j, file_paths_test[j]))
                print("----------------------------------------------------")
                cont+=1
    print("Registro do dataset_treino duplicado no dataset_teste: ", cont)

In [11]:
# Visualiza os dados de leitura
def visualiza_dados_leitura():
    plt.figure(figsize=(10, 10))
    for images, labels in train_dataset.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[labels[i]])
            plt.axis("off")

In [12]:
# Usando aumento de dados aleatórios somente no fit (treinamento)
data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        # keras.layers.experimental.preprocessing.RandomRotation(0.1), # retirar por conta dos cantos pretos
        keras.layers.experimental.preprocessing.RandomZoom(0.1),
        keras.layers.experimental.preprocessing.RandomContrast(0.2)
    ]
)

In [13]:
from tensorflow import expand_dims

# Testa visualização dos dados aumentados
def visualiza_dados_aumentados():
    for image, _ in train_dataset.take(1):
        plt.figure(figsize=(10, 10))
        first_image = image[0]
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            augmented_image = data_augmentation(expand_dims(first_image, 0))
            plt.imshow(augmented_image[0] / 255)
            plt.axis('off')

In [14]:
# Construir modelo
def build_model():
    
    # Freeze the base_model
    # Congela a base convolucional antes de compilar e treinar o modelo
    # Evita que os pesos em uma determinada camada sejam atualizados durante o treinamento
    base_model.trainable = False
    
    # Arquitetura do modelo básico
    # base_model.summary()

    # Adiciona o cabeçalho de classificação
    # Converter as features do shape `base_model.output_shape[1:] para vectores
    global_average_layer = keras.layers.GlobalAveragePooling2D()

    # Aplica uma camada densa para converter essas features em uma única previsão por imagem
    # Os números > 0.5 preveem a classe 1, os números <= 0.5 preveem a classe 0
    prediction_layer = keras.layers.Dense(1, activation="sigmoid", name='predictions') # Função de ativação sigmoid adicionada

    # Modelo encadeando as camadas de aumento de dados, reescalonamento, base_model e extrator de features
    img_shape = img_size + (3,)
    inputs = keras.Input(shape = img_shape)
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = keras.Model(inputs, outputs)    
    
    return model

In [15]:
from tensorflow_addons.metrics.cohens_kappa import CohenKappa
from tensorflow_addons.metrics.f_scores import F1Score

# Compile o modelo antes de treiná-lo
def compile_model(learning_rate):
    
    metricas = [  
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        F1Score(num_classes=2, threshold=0.5, average='micro', name='f1_score'),
        CohenKappa(num_classes=2, name='cohen_kappa')
    ]
    
    # Compilar modelo
    # Não especifiquei o batch_size, pois os dados já estão em conjuntos (batchs)
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss=keras.losses.BinaryCrossentropy(), # retirei o parâmetro from_logits=True para utilizar a função de ativação sigmoid
                  metrics=metricas) # A lista de métricas só funciona caso informe a função de ativação ao compilar o modelo
    
    # model.summary()

In [16]:
# Treinamento do modelo
def treinamento_model(qnt_epocas, epoca_inicial):
    history = model.fit(train_dataset,
                        epochs=qnt_epocas,
                        initial_epoch=epoca_inicial,
                        validation_data=validation_dataset,
                        verbose=0)
    
    return history

In [17]:
# Avaliação do modelo 
def avaliacao_model(dataset):
    # Não especifiquei o batch_size, pois os dados já estão em conjuntos (batchs)
    resultado = model.evaluate(dataset, verbose=0)
    
    return resultado

In [18]:
# Curvas de aprendizado da precisão / perda de treinamento e validação ao usar o modelo
def aprendizado_treinamento():
    
    metricas_graficos = ["loss", "accuracy", "precision", "recall", "f1_score", "cohen_kappa"]
          
    fig, ax = plt.subplots(len(metricas_graficos), 1, figsize=(10, len(metricas_graficos)*6))
    ax = ax.ravel()
    dados_x = np.arange(1, epocas_treinamento+1, 1)

    for i, met in enumerate(metricas_graficos):        
        ax[i].plot(dados_x, history.history[met], label='Training ' + met)
        ax[i].plot(dados_x, history.history["val_" + met], label='Validation ' + met)
        ax[i].set_title("Training and Validation %s in model %s" %(met, models[rede].name))
        ax[i].set_xlabel("epochs")
        ax[i].set_ylabel(met)
        
        if (i != 0): # loss function
            ax[i].legend(loc='lower right')
        else:
            ax[i].legend(loc='upper right')
    
    return metricas_graficos

In [19]:
# LIMIAR PARA AJUSTE FINO (OPCIONAL)
# Definir as camadas inferiores como não treináveis
def limiar_ajuste_fino():
    
    # Exibe a quantidade de camadas do modelo base
    # print("Número de camadas no modelo base: ", len(base_model.layers))

    # Ajuste fino desta camada em diante
    fine_tune_at = 100

    # Congele todas as camadas antes da camada 'fine_tune_at'
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

In [20]:
# Ajuste fino
def ajuste_fino():
    
    # Foi treinado apenas algumas camadas do modelo. 
    # Os pesos da rede pré-treinada não foram atualizados durante o treinamento.
    # Uma maneira de aumentar ainda mais o desempenho é treinar (ou "ajustar") os pesos das camadas superiores 
    # do modelo pré-treinado junto com o treinamento do classificador adicionado (camada de classificação adicionada)
    # Descongelar as camadas superiores do modelo (descongelar base_model)
    base_model.trainable = True
    
    # Limiar ajuste fino (OPCIONAL)
    limiar_ajuste_fino()
    
    # É necessário recompilar o modelo (para que essas alterações tenham efeito)
    # É importante usar uma taxa de aprendizado mais baixa neste estágio, 
    # pois está usando um modelo muito maior e deseja readaptar os pesos pré-treinados
    compile_model(base_learning_rate/10)
    
    # Retomar o treinamento melhorará sua precisão em alguns pontos percentuais
    # history.epoch[-1] é a última época do ultimo treinamento
    history_fine = treinamento_model(total_epocas, history.epoch[-1]+1)
    
    return history_fine

In [21]:
# Curvas de aprendizado da precisão / perda de treinamento e validação ao ajustar as últimas camadas do modelo
def aprendizado_ajuste_fino():
    
    fig, ax = plt.subplots(len(metricas_graficos), 1, figsize=(12, len(metricas_graficos)*6))
    ax = ax.ravel()
    dados_x = np.arange(1, total_epocas+1, 1)

    for i, met in enumerate(metricas_graficos):
        dados_treino = history.history[met] + history_fine.history[met]
        dados_validacao = history.history["val_" + met] + history_fine.history["val_" + met]
        
        ax[i].plot(dados_x, dados_treino, label='Training ' + met)
        ax[i].plot(dados_x, dados_validacao, label='Validation ' + met)
        ax[i].plot([epocas_treinamento, epocas_treinamento], plt.ylim(), label='Start Fine Tuning')
        ax[i].set_title("Training and Validation %s in model %s" %(met, models[rede].name))
        ax[i].set_xlabel("epochs")
        ax[i].set_ylabel(met)
        
        if (i != 0): # loss function
            ax[i].legend(loc='lower right')
        else:
            ax[i].legend(loc='upper right')

In [22]:
from tensorflow import nn, where

# Recupera um lote de imagens do conjunto de teste
def lote_imagens_teste():
    image_batch, label_batch = test_dataset.as_numpy_iterator().next()
    predictions = model.predict_on_batch(image_batch).flatten()

    # Aplique um sigmóide uma vez que nosso modelo retorna logits
    # predictions = nn.sigmoid(predictions) # Funcao sigmoid já aplicada
    predictions = where(predictions < 0.5, 0, 1) # Limiar

    print('Predictions:\n', predictions.numpy())
    print('Labels:\n', label_batch)

    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(image_batch[i].astype("uint8"))
        plt.title(class_names[predictions[i]])
        plt.axis("off")

In [23]:
def insere_novas_linhas(df_resultados):
    # Insere linhas no data_frame
    nova_linha = {}
    nova_linha["model"] = models[rede].name

    for nome, valor in zip(model.metrics_names, resultado):
        nova_linha[nome] = valor

    df_resultados = df_resultados.append(nova_linha, ignore_index=True)
    
    return df_resultados

In [24]:
# Média dos resultados
def resultados_mean():
    df_mean = pd.DataFrame(columns = colunas_dataframe)

    for i in range(len(models)):
        lista_media = []
        lista_media.append(models[i].name)
        lista_media = lista_media + list(df_resultados.loc[df_resultados['model'] == models[i].name].mean())

        series_mean = pd.Series(lista_media, index = df_mean.columns)
        df_mean = df_mean.append(series_mean, ignore_index=True)
        
    return df_mean

In [25]:
# Desvio Padrão dos resultados
def resultados_std():
    df_std = pd.DataFrame(columns = colunas_dataframe)

    for i in range(len(models)):
        lista_std = []
        lista_std.append(models[i].name)
        lista_std = lista_std + list(df_resultados.loc[df_resultados['model'] == models[i].name].std())

        series_std = pd.Series(lista_std, index = df_std.columns)
        df_std = df_std.append(series_std, ignore_index=True)
        
    return df_std

# Execução das funções

In [26]:
# Verifica imagens repetidas no dataset leishmaniose
img_repetidas()

Quantidade de registros duplicados:  0


In [27]:
# Cria data frame de resultados
colunas_dataframe = ['model', 'loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'f1_score', 'cohen_kappa']
df_resultados = pd.DataFrame(columns = colunas_dataframe)

In [28]:
for iteracao in range(iteracoes_algoritomo):

    # Divide as imagens em treino, teste e validação em cada iteração
    split_imagens()

    for rede in range(len(models)): # Todos os modelos

        print('\nRunning model {}'.format(models[rede].name))

        # Modelo base
        base_model, img_size, preprocess_input = modelo_base(rede)

        # Leitura dos dados
        train_dataset, validation_dataset, test_dataset, class_names, file_paths_train, file_paths_test = leitura_dados(img_size)

        # Verifica duplicação na divisão do dataset entre treino e teste
        # duplicacao_treino_teste()

        # Visualiza dados leitura
        # visualiza_dados_leitura()

        # Testa visualização dos dados aumentados
        # visualiza_dados_aumentados()

        # Construir modelo
        model = build_model()

        # Complilar modelo
        compile_model(base_learning_rate)

        # Avaliação do modelo antes de treinar (conjunto de validação)
        avaliacao_model(validation_dataset)

        # Treinamento do modelo
        history = treinamento_model(epocas_treinamento, 0)

        # Curvas de aprendizado da precisão / perda de treinamento e validação ao usar o modelo
        # metricas_graficos = aprendizado_treinamento()

        # Ajuste fino
        history_fine = ajuste_fino()

        # Curvas de aprendizado da precisão / perda de treinamento e validação ao ajustar as últimas camadas do modelo
        # aprendizado_ajuste_fino()

        # Avaliação do modelo em novos dados (conjunto de teste)
        resultado = avaliacao_model(test_dataset)

        # Insere novas linhas no dataframe de resultados
        df_resultados = insere_novas_linhas(df_resultados)

        # Recupera um lote de imagens do conjunto de teste
        # lote_imagens_teste()
        
    print("\n\nIteração: ", iteracao + 1)
    print(df_resultados.tail(len(models)))
    print("\n\n")

Copying files: 151 files [00:00, 915.62 files/s] 


Running model inception_v3


Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model xception
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model inception_resnet_v2
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model NASNet
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model resnet152v2
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.


Copying files: 73 files [00:00, 565.65 files/s]



Iteração:  1
                 model      loss    tp   fp    tn   fn  accuracy  precision  \
0         inception_v3  0.149787  21.0  0.0  22.0  4.0  0.914894   1.000000   
1             xception  0.346701  21.0  4.0  18.0  4.0  0.829787   0.840000   
2  inception_resnet_v2  0.278509  20.0  0.0  22.0  5.0  0.893617   1.000000   
3               NASNet  0.075729  25.0  1.0  21.0  0.0  0.978723   0.961538   
4          resnet152v2  0.196882  23.0  1.0  21.0  2.0  0.936170   0.958333   

   recall  f1_score  cohen_kappa  
0    0.84  0.913043     0.830935  
1    0.84  0.840000     0.658182  
2    0.80  0.888889     0.789238  
3    1.00  0.980392     0.957156  
4    0.92  0.938775     0.872167  





...


Running model inception_v3
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model xception
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model inception_resnet_v2
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model NASNet
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model resnet152v2
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.


Copying files: 73 files [00:00, 627.61 files/s]



Iteração:  9
                  model          loss    tp   fp    tn   fn  accuracy  \
40         inception_v3  2.035590e-01  24.0  0.0  22.0  1.0  0.978723   
41             xception  1.920178e-01  23.0  0.0  22.0  2.0  0.957447   
42  inception_resnet_v2  6.227274e-07  25.0  0.0  22.0  0.0  1.000000   
43               NASNet  8.204506e-05  25.0  0.0  22.0  0.0  1.000000   
44          resnet152v2  6.131795e-02  24.0  0.0  22.0  1.0  0.978723   

    precision  recall  f1_score  cohen_kappa  
40        1.0    0.96  0.979592     0.957389  
41        1.0    0.92  0.958333     0.915009  
42        1.0    1.00  1.000000     1.000000  
43        1.0    1.00  1.000000     1.000000  
44        1.0    0.96  0.979592     0.957389  





Copying files: 151 files [00:00, 729.47 files/s]



Running model inception_v3
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model xception
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model inception_resnet_v2
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model NASNet
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.

Running model resnet152v2
Found 90 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 47 files belonging to 2 classes.


Iteração:  10
                  model      loss    tp   fp    tn   fn  accuracy  precision  \
45         inception_v3  0.000443  25.0  0.0  22.0  0.0  1.000000        1.0   
46             xception  0.119305  23.0  0.0  22.0  2.0  0.957447        1.0   
47  inception_resnet_v2

In [34]:
# Resultados de todas as iterações
df_resultados.to_csv('./df_resultados.csv', index=False, header=True)
df_resultados

,model,loss,tp,fp,tn,fn,accuracy,precision,recall,f1_score,cohen_kappa
0,inception_v3,1.497869e-01,21.0,0.0,22.0,4.0,0.914894,1.000000,0.84,0.913043,0.830935
1,xception,3.467013e-01,21.0,4.0,18.0,4.0,0.829787,0.840000,0.84,0.840000,0.658182
2,inception_resnet_v2,2.785090e-01,20.0,0.0,22.0,5.0,0.893617,1.000000,0.80,0.888889,0.789238
3,NASNet,7.572898e-02,25.0,1.0,21.0,0.0,0.978723,0.961538,1.00,0.980392,0.957156
4,resnet152v2,1.968820e-01,23.0,1.0,21.0,2.0,0.936170,0.958333,0.92,0.938775,0.872167
5,inception_v3,4.569483e-02,24.0,0.0,22.0,1.0,0.978723,1.000000,0.96,0.979592,0.957389
6,xception,1.960494e-01,22.0,0.0,22.0,3.0,0.936170,1.000000,0.88,0.936170,0.872858
7,inception_resnet_v2,1.081482e-01,24.0,1.0,21.0,1.0,0.957447,0.960000,0.96,0.960000,0.914545
8,NASNet,4.599806e-03,25.0,0.0,22.0,0.0,1.000000,1.000000,1.00,1.000000,1.000000
9,resnet152v2,7.544153e-02,24.0,0.0,22.0,1.0,0.978723,1.000000,0.96,0.979592,0.957389


In [37]:
# Média dos resultados
media_resultados = resultados_mean()
media_resultados.to_csv('./media_resultados.csv', index=False, header=True)
media_resultados

,model,loss,tp,fp,tn,fn,accuracy,precision,recall,f1_score,cohen_kappa
0,inception_v3,0.070085,24.0,0.0,22.0,1.0,0.978723,1.000000,0.960,0.978889,0.957573
1,xception,0.168681,23.3,0.9,21.1,1.7,0.944681,0.965548,0.932,0.947520,0.889022
2,inception_resnet_v2,0.053951,24.4,0.2,21.8,0.6,0.982979,0.992154,0.976,0.982928,0.966094
3,NASNet,0.046790,24.7,0.1,21.9,0.3,0.991489,0.996154,0.988,0.991917,0.982932
4,resnet152v2,0.087672,24.0,0.1,21.9,1.0,0.976596,0.995833,0.960,0.977381,0.953174


In [38]:
# Desvio padrão dos resultados
desvio_resultados = resultados_std()
desvio_resultados.to_csv('./desvio_resultados.csv', index=False, header=True)
desvio_resultados

,model,loss,tp,fp,tn,fn,accuracy,precision,recall,f1_score,cohen_kappa
0,inception_v3,0.076629,1.333333,0.000000,0.000000,1.333333,0.028369,0.000000,0.053333,0.028694,0.056432
1,xception,0.100043,1.251666,1.663330,1.663330,1.251666,0.051339,0.063135,0.050067,0.047616,0.103316
2,inception_resnet_v2,0.093782,1.577621,0.421637,0.421637,1.577621,0.034454,0.016545,0.063105,0.035615,0.068364
3,NASNet,0.074915,0.483046,0.316228,0.316228,0.483046,0.010987,0.012163,0.019322,0.010438,0.022034
4,resnet152v2,0.079569,0.816497,0.316228,0.316228,0.816497,0.021158,0.013176,0.032660,0.020468,0.042327


# Grad-CAM

In [32]:
# Stop execução
glob.asdsad(asdas)

AttributeError: 'function' object has no attribute 'asdsad'

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
import cv2

class GradCAM:
	def __init__(self, model, classIdx, inner_model=None, layerName=None):
		# store the model, the class index used to measure the class
		# activation map, and the layer to be used when visualizing
		# the class activation map
		self.model = model
		self.classIdx = classIdx
		self.inner_model = inner_model
		if self.inner_model == None:
			self.inner_model = model
		self.layerName = layerName
		# if the layer name is None, attempt to automatically find
		# the target output layer
		if self.layerName is None:
			self.layerName = self.find_target_layer()
            
	def find_target_layer(self):
		# attempt to find the final convolutional layer in the network
		# by looping over the layers of the network in reverse order
		for layer in reversed(self.model.layers):
			# check to see if the layer has a 4D output
			if len(layer.output_shape) == 4:
				return layer.name
		# otherwise, we could not find a 4D layer so the GradCAM
		# algorithm cannot be applied
		raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")
        
	def compute_heatmap(self, image, eps=1e-8):
		# construct our gradient model by supplying (1) the inputs
		# to our pre-trained model, (2) the output of the (presumably)
		# final 4D layer in the network, and (3) the output of the
		# softmax activations from the model
		gradModel = Model(inputs=[self.inner_model.inputs], outputs=[self.inner_model.get_layer(self.layerName).output, self.inner_model.output])
        
		# record operations for automatic differentiation
		with tf.GradientTape() as tape:
			# cast the image tensor to a float-32 data type, pass the
			# image through the gradient model, and grab the loss
			# associated with the specific class index
			inputs = tf.cast(image, tf.float32)
			(convOutputs, predictions) = gradModel(inputs)
			loss = predictions[:, self.classIdx]
		# use automatic differentiation to compute the gradients
		grads = tape.gradient(loss, convOutputs)
        
		# compute the guided gradients
		castConvOutputs = tf.cast(convOutputs > 0, "float32")
		castGrads = tf.cast(grads > 0, "float32")
		guidedGrads = castConvOutputs * castGrads * grads
		# the convolution and guided gradients have a batch dimension
		# (which we don't need) so let's grab the volume itself and
		# discard the batch
		convOutputs = convOutputs[0]
		guidedGrads = guidedGrads[0]
        
		# compute the average of the gradient values, and using them
		# as weights, compute the ponderation of the filters with
		# respect to the weights
		weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
		cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)
        
		# grab the spatial dimensions of the input image and resize
		# the output class activation map to match the input image
		# dimensions
		(w, h) = (image.shape[2], image.shape[1])
		heatmap = cv2.resize(cam.numpy(), (w, h))
		# normalize the heatmap such that all values lie in the range
		# [0, 1], scale the resulting values to the range [0, 255],
		# and then convert to an unsigned 8-bit integer
		numer = heatmap - np.min(heatmap)
		denom = (heatmap.max() - heatmap.min()) + eps
		heatmap = numer / denom
		heatmap = (heatmap * 255).astype("uint8")
		# return the resulting heatmap to the calling function
		return heatmap

	def overlay_heatmap(self, heatmap, image, alpha=0.5,
		colormap=cv2.COLORMAP_VIRIDIS):
		# apply the supplied color map to the heatmap and then
		# overlay the heatmap on the input image
		heatmap = cv2.applyColorMap(heatmap, colormap)
		output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
		# return a 2-tuple of the color mapped heatmap and the output,
		# overlaid image
		return (heatmap, output)


In [ ]:
import cv2

model_layer = "inception_v3"
last_conv_layer_name = "conv2d_93"
cam = GradCAM(model, None, inner_model=model.get_layer(model_layer), layerName=last_conv_layer_name)

def grad_cam_teste():
    image_batch, label_batch = test_dataset.as_numpy_iterator().next()
    predictions = model.predict_on_batch(image_batch).flatten()

    predictions = where(predictions < 0.5, 0, 1) # Limiar

    print('Predictions:\n', predictions.numpy())
    print('Labels:\n', label_batch)

    plt.figure(figsize=(40, 80))
    for i in range(10):        
        image = image_batch[i].astype("uint8")
        image_cv2 = cv2.resize(image, img_size)
        
        image = np.expand_dims(image, axis=0)
        label = class_names[predictions[i]]
        
        # load the input image from disk (in Keras/TensorFlow format) and
        # preprocess it
        heatmap = cam.compute_heatmap(image)

        # resize the resulting heatmap to the original input image dimensions
        # and then overlay heatmap on top of the image
        heatmap = cv2.resize(heatmap, img_size)
        (heatmap, output) = cam.overlay_heatmap(heatmap, image_cv2, alpha=0.5)

        # draw the predicted label on the output image
        # cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1)
        cv2.putText(output, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        # display the original image and resulting heatmap and output image

        # to our screen
        output = np.hstack([image_cv2, heatmap, output])
        # cv2.imshow("Output", output)
        # cv2.waitKey(0)
        
        ax = plt.subplot(10, 1, i + 1)
        plt.imshow(output)
        plt.axis("off")

grad_cam_teste()